<a href="https://colab.research.google.com/github/junyamahira/ReinforcementLearning_Prac/blob/master/DQN_MountainCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# git clone 
- これかpipのどちらかを実行すればよい
- keras-rlの中に入り、setup.pyを実行する

In [0]:
!git clone https://github.com/keras-rl/keras-rl.git

Cloning into 'keras-rl'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1710 (delta 3), reused 7 (delta 2), pack-reused 1698
Receiving objects: 100% (1710/1710), 1.38 MiB | 19.35 MiB/s, done.
Resolving deltas: 100% (1056/1056), done.


In [0]:
!ls keras-rl

assets		 examples	    mkdocs.yml	rl	   tests
CONTRIBUTING.md  ISSUE_TEMPLATE.md  pytest.ini	setup.cfg  utils
docs		 LICENSE	    README.md	setup.py


In [0]:
!cd keras-rl
!ls

keras-rl  sample_data


# pip

In [1]:
pip install keras-rl

    100% |████████████████████████████████| 40kB 5.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl


# import

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
import gym
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.


In [0]:
env = gym.make("MountainCar-v0")
nb_actions = env.action_space.n

# モデル

In [4]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
memory = SequentialMemory(limit=50000, window_length=1)

policy = EpsGreedyQPolicy(eps=0.001)
dqn = DQNAgent(model=model, nb_actions=nb_actions,gamma=0.99, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

history = dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)


Training for 50000 steps ...
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


   200/50000: episode: 1, duration: 1.430s, episode steps: 200, steps per second: 140, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.885 [0.000, 2.000], mean observation: -0.233 [-1.200, 0.092], loss: 0.200101, mean_absolute_error: 0.638835, mean_q: -0.618114
   400/50000: episode: 2, duration: 0.611s, episode steps: 200, steps per second: 327, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.770 [0.000, 2.000], mean observation: -0.275 [-0.704, 0.018], loss: 0.002689, mean_absolute_error: 1.501911, mean_q: -2.210117
   600/50000: episode: 3, duration: 0.597s, episode steps: 200, steps per second: 335, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.730 [0.000, 2.000], mean observation: -0.284 [-0.737, 0.023], loss: 0.017980, mean_absolute_error: 2.541771, mean_q: -3.754195
   800/50000: episode: 4, duration: 0.568s, episode steps: 200, steps per second: 352, episode reward: -200.000, mean rew

In [7]:
dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...


NameError: ignored